In [2]:
import pandas as pd
import numpy as np
from pymorphy2 import MorphAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans

In [3]:
df= pd.read_csv('dataset_users_query.csv')
df.head()

,query
0,фарпост работа владивосток
1,кофейни вакансии
2,работа разнорабочие часовой
3,личный водитель на день
4,работа от работодателя персональный водитель


In [4]:
df['Фраза_по_словам'] = df['query'].str.lower().str.split()
df.head()

,query,Фраза_по_словам
0,фарпост работа владивосток,"[фарпост, работа, владивосток]"
1,кофейни вакансии,"[кофейни, вакансии]"
2,работа разнорабочие часовой,"[работа, разнорабочие, часовой]"
3,личный водитель на день,"[личный, водитель, на, день]"
4,работа от работодателя персональный водитель,"[работа, от, работодателя, персональный, водит..."


## Очистка датасета от предлогов, чисел и отдельных букв

In [5]:
predlogi = pd.read_excel('spisok_predlogov.xlsx')
predlogi = predlogi['Unnamed: 0'].dropna()
unique_values = predlogi.str.lower().unique()
unique_values = unique_values.tolist()
predlogi = unique_values
print(predlogi)

['без', 'безо', 'близ', 'в', 'во', 'вместо', 'вне', 'для', 'до', 'за', 'из', 'из-за', 'изо', 'из-под', 'изпод', 'под', 'к', 'ко', 'кроме', 'между', 'меж', 'на', 'над', 'о', 'об', 'обо', 'от', 'ото', 'перед', 'передо', 'пред', 'предо', 'по', 'подо', 'при', 'про', 'ради', 'с', 'со', 'сквозь', 'среди', 'у', 'через', 'чрез', 'и', 'или', 'c']


In [6]:
def find_predlogi(splitted_phrase):
    for word in splitted_phrase:
        if word in predlogi:
            splitted_phrase.remove(word)
    return splitted_phrase

In [7]:
df['Фраза_по_словам'] = df['Фраза_по_словам'].apply(find_predlogi)

In [8]:
def find_num(splitted_phrase):
    return [word for word in splitted_phrase if not any(char.isdigit() for char in word)]

In [9]:
df['Фраза_по_словам'] = df['Фраза_по_словам'].apply(find_num)

In [10]:
def find_word(splitted_phrase):
    return [word for word in splitted_phrase if len(word) > 1 or not word.isalpha()]

In [11]:
df['Фраза_по_словам'] = df['Фраза_по_словам'].apply(find_word)

In [12]:
df['Фраза_по_словам'] = df['Фраза_по_словам'].apply(lambda x: ' '.join(x))
df.head(100)

,query,Фраза_по_словам
0,фарпост работа владивосток,фарпост работа владивосток
1,кофейни вакансии,кофейни вакансии
2,работа разнорабочие часовой,работа разнорабочие часовой
3,личный водитель на день,личный водитель день
4,работа от работодателя персональный водитель,работа работодателя персональный водитель
...,...,...
95,работа в автомойке,работа автомойке
96,работа в ночную смену для девушек,работа ночную смену девушек
97,работа водителем курьером на авто работодателя,работа водителем курьером авто работодателя
98,работа водитель курьер на легковом автомобиле,работа водитель курьер легковом автомобиле


## Лемматизация

In [ ]:
df_work = df.head(3000).copy()

from pymystem3 import Mystem
m = Mystem()
def lemmatize_sentence(text):
    lemmas = m.lemmatize(text)
    return "".join(lemmas).strip()

df_work['Леммы'] = [lemmatize_sentence(x) for x in df_work['Фраза_по_словам']]
df_work.head()

In [21]:
df_work = pd.read_excel('output_work.xlsx',index_col = 'id')
df_work.head()

,query,Фраза_по_словам,Леммы
id,,,
0,фарпост работа владивосток,фарпост работа владивосток,фарпост работа владивосток
1,кофейни вакансии,кофейни вакансии,кофейня вакансия
2,работа разнорабочие часовой,работа разнорабочие часовой,работа разнорабочий часовой
3,личный водитель на день,личный водитель день,личный водитель день
4,работа от работодателя персональный водитель,работа работодателя персональный водитель,работа работодатель персональный водитель
